In [4]:
import import_ipynb
import os
import pdfplumber
# from Read_pdf import get_text
from pathlib import Path
import PySimpleGUI as sg
import time

def getFileList(dir ,**kwargs):
    kwargs.setdefault('filetype', '.pdf')
    kwargs.setdefault('include_chars', None)
    files = os.listdir(dir)
    path_list=[]
    
    for file in files:
        full_path = os.path.join(dir, file)
        if os.path.isfile(full_path) and os.path.splitext(file)[-1] == kwargs['filetype']:
            if kwargs['include_chars'] == None:
                li=[]
                li.append(os.path.basename(full_path))
                li.append(os.path.dirname(full_path))
                path_list.append(li)
            elif kwargs['include_chars'] in os.path.splitext(file)[0]:
                li=[]
                li.append(os.path.basename(full_path))
                li.append(os.path.dirname(full_path))
                path_list.append(li)

    return path_list

def walkpath(**kwargs):
    kwargs.setdefault('filetype', '.pdf')
    kwargs.setdefault('path', str(Path.home()) + r'\AppData\Local\Microsoft\Windows\INetCache\IE\\')
    kwargs.setdefault('include_chars', None)
    path_list=[]
    
    for root, dirs, files in os.walk(kwargs['path']):
        for file in files:
            full_path = os.path.join(root, file)
            if os.path.splitext(full_path)[-1] == kwargs['filetype']:
                if kwargs['include_chars'] == None:
                    li=[]
                    li.append(os.path.basename(full_path))
                    li.append(os.path.dirname(full_path))
                    path_list.append(li)
                elif kwargs['include_chars'] in os.path.splitext(file)[0]:
                    li=[]
                    li.append(os.path.basename(full_path))
                    li.append(os.path.dirname(full_path))
                    path_list.append(li)
    return path_list

def getCasename(path, pos_name, pos_sn):
    
    pdf_1 = pdfplumber.open(path)
    p0 = pdf_1.pages[0]
    text = []
    text.append(p0.within_bbox((pos_name['x0'],pos_name['top'],pos_name['x1'],pos_name['bottom'])).extract_text())
    
    if text[0] != None:
        sn = p0.within_bbox((pos_sn['x0'],pos_sn['top'],pos_sn['x1'],pos_sn['bottom'])).extract_text()
        if sn == None:
            sn = ''

        text.append(sn)

    return text

def append_pdftext(li, **kwargs):
    kwargs.setdefault('pos_name', {'x0':29.280, 'top':135.104, 'x1':393.600, 'bottom':152.960})#預設工程名稱的位置
    kwargs.setdefault('pos_sn',  {'x0':81.320, 'top':91.744, 'x1':93.320, 'bottom':109.600})#預設次別的位置
    kwargs.setdefault('title', '指定資料夾')
    
    #progress bar creating with pySimpleGui
#     sg.theme('LightGrey3')
    
    layout = [[sg.Text('擷取{}內的PDF案件名稱及次別中...'.format(kwargs['title']), font ='JhengHei 9')],
          [sg.ProgressBar(len(li), orientation='h', size=(20, 15), key='progressbar')]]

    window = sg.Window('資料讀取中', layout, ttk_theme=sg.THEME_VISTA)
    progress_bar = window['progressbar']
    event, values = window.read(timeout=10)

    i = len(li) -1
    
    for j in range(len(li)):
        filepath = os.path.join(li[i][1], li[i][0])
        case_info = getCasename(filepath, kwargs['pos_name'], kwargs['pos_sn'])
        
        
        if case_info[0] != None:#判斷是不是案件pdf
            li[i].append(case_info[0] + '-' + case_info[1])
        else:
            del li[i]
                
        i-=1
        progress_bar.UpdateBar(j)
    
    window.close()
    return li

if __name__=='__main__':
    start = time.time()
    r =getFileList('d:/', filetype = '.pdf')
    r = append_pdftext(r, title = 'D:/')
    end = time.time()
#     print(end - start)
    print('讀取資料夾內檔案並擷取pdf檔工程名稱:')
    for item in r:
        print(item)
    
    tempdir = str(Path.home()) + r'\AppData\Local\Microsoft\Windows\INetCache\\'
    start = time.time()
    r = walkpath()
    r = append_pdftext(r, title = 'ERP產出')
    print('走遍路徑內資料及其檔案及並擷取其pdf檔工程名稱:')
    print(r)
    end = time.time()
    print(end - start)

讀取資料夾內檔案並擷取pdf檔工程名稱:
['B1.pdf', 'd:/', '宁波PABS厂成品硬性围篱改善-00']
['N1.pdf', 'd:/', 'PVC廠屋面彩鋼更換及漏水修繕工程-00']
['N2-w.pdf', 'd:/', '昆山CCL三廠配料室T-316至含浸配管-00']
['N2.pdf', 'd:/', '昆山CCL三廠配料室T-316至含浸配管-00']
['N3.pdf', 'd:/', '苯酚厂废水在线监测委外检测工程-00']
['N4-w.pdf', 'd:/', '寧波PTA廠氧化塔逆止閥螺栓拆除-00']
['N4.pdf', 'd:/', '寧波PTA廠氧化塔逆止閥螺栓拆除-00']
['N5-w.pdf', 'd:/', '苯酚厂K-621废液焚烧设备检修工程-00']
['N5.pdf', 'd:/', '苯酚厂K-621废液焚烧设备检修工程-00']
['S1.pdf', 'd:/', '煉鋼運轉兩處倉庫新增照明-00']
['T1.pdf', 'd:/', '一二廠衛生間改善-03']
['T2.pdf', 'd:/', '寧波工事處辦公樓外墻面改善工程-01']
['T3-w.pdf', 'd:/', '寧波PTA廠氫氣回收改善保溫工程-02']
['T3.pdf', 'd:/', '寧波PTA廠氫氣回收改善保溫工程-02']
['T4-w.pdf', 'd:/', '厂区篮球场及倒班楼楼梯墙裙修复-01']
['T4.pdf', 'd:/', '厂区篮球场及倒班楼楼梯墙裙修复-01']
['T5-w.pdf', 'd:/', '厂区篮球场及倒班楼楼梯墙裙修复-01']
['T5.pdf', 'd:/', '厂区篮球场及倒班楼楼梯墙裙修复-01']
['tw1.pdf', 'd:/', '3A增設冷凍機備機配管工程-01']
['tw2.pdf', 'd:/', 'EG-1 碳酸鹽熱交換器增設改善保溫-01']
[['N4.pdf', 'C:\\Users\\spp43\\AppData\\Local\\Microsoft\\Windows\\INetCache\\IE']]
走遍路徑內資料及其檔案及並擷取其pdf檔工程名稱:
[['N4.pdf', 'C:\\Users\\spp43\\AppData\\Lo